# Portfolio Workflow

In [1]:
from platform import python_version
import time
from datetime import datetime, timedelta
import os
import pandas as pd
import pandas_datareader as pdr

import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, 'tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import trading_factors as alpha_factors
importlib.reload(alpha_factors)
import utils
importlib.reload(utils)
import nonoverlapping_estimator as ai_estimator
importlib.reload(ai_estimator)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')
print(f'Pandas Data Reader version: {pdr.__version__}')

Sci-Kit version: 0.24.1
Sci-Kit version: 0.24.1
Python version: 3.8.8
Pandas version: 1.2.4
Pandas Data Reader version: 0.10.0


In [2]:
from pathlib import Path

# Make sure we have a data directory
Path('./data').mkdir(parents=True, exist_ok=True) 

# Which account are we interested in
masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories_yahoo.csv'

# Stage 1: Generate Stock Universe

- Gather stocks from specific criteria (SP500 top 50...)
- Use stock sentiment to select stocks
- Gather price histories

## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

We will obtain 5 years of price histories. In the end this will provide us with 2 years of factor data since some of the factors are based on 1 year returns.

In [3]:
snp_500_stocks = utils.get_snp500()
stock_universe = utils.reduce_universe_by_sentiment(set(snp_500_stocks.index.to_list()))

Number of stocks in universe: 505


Tickers:   0%|          | 0/505 [00:00<?, ?Finvis Postings/s]

News Tables:   0%|          | 0/503 [00:00<?, ?News Table Items/s]

Mean Sentiment: 2.952871734087711 with a standared deviation of: 1.6873899497225768 providing a cutoff of: 1.2654817843651343
New number of stocks in universe: 419


In [4]:
from dateutil.relativedelta import relativedelta
number_of_years = 5
start = datetime.today() - relativedelta(years = number_of_years)
yahoo = pdr.yahoo.daily.YahooDailyReader(symbols=stock_universe, start=start, adjust_price=True, interval='d', get_actions=False, adjust_dividends=True)
price_histories = yahoo.read()
yahoo.close()
price_histories.to_csv(price_histories_file_name, index=True)

In [5]:
price_histories

Attributes Adj_Ratio                                                         \
Symbols            A       AAP      ABBV       ABC       ABT       ACN ADBE   
Date                                                                          
2017-02-27  0.960623  0.971871  0.794469  0.917661  0.917953  0.922028  1.0   
2017-02-28  0.960623  0.971871  0.794469  0.917661  0.917953  0.922028  1.0   
2017-03-01  0.960623  0.971872  0.794469  0.917661  0.917953  0.922028  1.0   
2017-03-02  0.960623  0.971871  0.794469  0.917660  0.917953  0.922028  1.0   
2017-03-03  0.960623  0.971871  0.794469  0.917661  0.917953  0.922028  1.0   
...              ...       ...       ...       ...       ...       ...  ...   
2022-02-18  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   
2022-02-22  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   
2022-02-23  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   
2022-02-24  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   
2022-02-25  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   

Attributes                                ...     Volume             \
Symbols          ADI       ADM       ADP  ...        WTW         WY   
Date                                      ...                         
2017-02-27  0.906076  0.860876  0.899408  ...   422800.0  3649200.0   
2017-02-28  0.906076  0.860876  0.899407  ...   717000.0  3792500.0   
2017-03-01  0.906076  0.860876  0.899407  ...   876200.0  4320500.0   
2017-03-02  0.906076  0.860876  0.899407  ...   805500.0  4477500.0   
2017-03-03  0.906076  0.860876  0.899407  ...   531200.0  3239200.0   
...              ...       ...       ...  ...        ...        ...   
2022-02-18  0.995143  1.000000  1.000000  ...  1072100.0  4299000.0   
2022-02-22  0.995144  1.000000  1.000000  ...  2037800.0  5509800.0   
2022-02-23  0.995143  1.000000  1.000000  ...   861900.0  5515800.0   
2022-02-24  1.000000  1.000000  1.000000  ...   846600.0  5911100.0   
2022-02-25  1.000000  1.000000  1.000000  ...  1167700.0  4569700.0   

Attributes                                                                   \
Symbols          WYNN        XEL       XRAY        YUM        ZBH      ZBRA   
Date                                                                          
2017-02-27  1694400.0  2334600.0  1611100.0  1816000.0  1577900.0  648000.0   
2017-02-28  2094100.0  2512800.0  2533400.0  2681400.0  1244200.0  424500.0   
2017-03-01  6088500.0  3383100.0  1998800.0  2311500.0  1101300.0  602600.0   
2017-03-02  2754100.0  2551200.0   972900.0  2176000.0   772200.0  390700.0   
2017-03-03  1782100.0  2174200.0  1234200.0  2170400.0   708100.0  250300.0   
...               ...        ...        ...        ...        ...       ...   
2022-02-18  3007100.0  3664200.0  1804100.0  2359400.0  1198400.0  536200.0   
2022-02-22  4338800.0  3556500.0  1880500.0  1863800.0  2082300.0  549600.0   
2022-02-23  2156100.0  3701500.0  1463500.0  2272500.0  1433000.0  569000.0   
2022-02-24  4455400.0  4720400.0  2101300.0  2447600.0  1427800.0  508700.0   
2022-02-25  2604300.0  4985000.0  2660000.0  1498200.0  1593800.0  440600.0   

Attributes                        
Symbols          ZION        ZTS  
Date                              
2017-02-27  2298900.0  4220600.0  
2017-02-28  2485700.0  4106300.0  
2017-03-01  3320700.0  3581100.0  
2017-03-02  2925900.0  3299900.0  
2017-03-03  2686400.0  2483600.0  
...               ...        ...  
2022-02-18  1701600.0  2558900.0  
2022-02-22  1223400.0  2347300.0  
2022-02-23  1148800.0  2070600.0  
2022-02-24  2880200.0  2903200.0  
2022-02-25  1941000.0  1742600.0  

[1260 rows x 2514 columns]

In [6]:
test_read = pd.read_csv(price_histories_file_name, header=[0, 1], index_col=[0], low_memory=False)

In [7]:
test_read

Attributes Adj_Ratio                                                         \
Symbols            A       AAP      ABBV       ABC       ABT       ACN ADBE   
Date                                                                          
2017-02-27  0.960623  0.971871  0.794469  0.917661  0.917953  0.922028  1.0   
2017-02-28  0.960623  0.971871  0.794469  0.917661  0.917953  0.922028  1.0   
2017-03-01  0.960623  0.971872  0.794469  0.917661  0.917953  0.922028  1.0   
2017-03-02  0.960623  0.971871  0.794469  0.917660  0.917953  0.922028  1.0   
2017-03-03  0.960623  0.971871  0.794469  0.917661  0.917953  0.922028  1.0   
...              ...       ...       ...       ...       ...       ...  ...   
2022-02-18  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   
2022-02-22  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   
2022-02-23  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   
2022-02-24  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   
2022-02-25  1.000000  1.000000  1.000000  1.000000  1.000000  1.000000  1.0   

Attributes                                ...     Volume             \
Symbols          ADI       ADM       ADP  ...        WTW         WY   
Date                                      ...                         
2017-02-27  0.906076  0.860876  0.899408  ...   422800.0  3649200.0   
2017-02-28  0.906076  0.860876  0.899407  ...   717000.0  3792500.0   
2017-03-01  0.906076  0.860876  0.899407  ...   876200.0  4320500.0   
2017-03-02  0.906076  0.860876  0.899407  ...   805500.0  4477500.0   
2017-03-03  0.906076  0.860876  0.899407  ...   531200.0  3239200.0   
...              ...       ...       ...  ...        ...        ...   
2022-02-18  0.995143  1.000000  1.000000  ...  1072100.0  4299000.0   
2022-02-22  0.995144  1.000000  1.000000  ...  2037800.0  5509800.0   
2022-02-23  0.995143  1.000000  1.000000  ...   861900.0  5515800.0   
2022-02-24  1.000000  1.000000  1.000000  ...   846600.0  5911100.0   
2022-02-25  1.000000  1.000000  1.000000  ...  1167700.0  4569700.0   

Attributes                                                                   \
Symbols          WYNN        XEL       XRAY        YUM        ZBH      ZBRA   
Date                                                                          
2017-02-27  1694400.0  2334600.0  1611100.0  1816000.0  1577900.0  648000.0   
2017-02-28  2094100.0  2512800.0  2533400.0  2681400.0  1244200.0  424500.0   
2017-03-01  6088500.0  3383100.0  1998800.0  2311500.0  1101300.0  602600.0   
2017-03-02  2754100.0  2551200.0   972900.0  2176000.0   772200.0  390700.0   
2017-03-03  1782100.0  2174200.0  1234200.0  2170400.0   708100.0  250300.0   
...               ...        ...        ...        ...        ...       ...   
2022-02-18  3007100.0  3664200.0  1804100.0  2359400.0  1198400.0  536200.0   
2022-02-22  4338800.0  3556500.0  1880500.0  1863800.0  2082300.0  549600.0   
2022-02-23  2156100.0  3701500.0  1463500.0  2272500.0  1433000.0  569000.0   
2022-02-24  4455400.0  4720400.0  2101300.0  2447600.0  1427800.0  508700.0   
2022-02-25  2604300.0  4985000.0  2660000.0  1498200.0  1593800.0  440600.0   

Attributes                        
Symbols          ZION        ZTS  
Date                              
2017-02-27  2298900.0  4220600.0  
2017-02-28  2485700.0  4106300.0  
2017-03-01  3320700.0  3581100.0  
2017-03-02  2925900.0  3299900.0  
2017-03-03  2686400.0  2483600.0  
...               ...        ...  
2022-02-18  1701600.0  2558900.0  
2022-02-22  1223400.0  2347300.0  
2022-02-23  1148800.0  2070600.0  
2022-02-24  2880200.0  2903200.0  
2022-02-25  1941000.0  1742600.0  

[1260 rows x 2514 columns]